In [ ]:
# ==========================
# 2FA AUTHENTICATION SYSTEM
# ==========================

!pip install -q pyotp qrcode pillow bcrypt

import pyotp
import qrcode
import bcrypt
import sqlite3
import json
import secrets
from datetime import datetime, timedelta
from IPython.display import display
import time

# ========================
# DATABASE INITIALIZATION
# ========================

def init_database():
    conn = sqlite3.connect("secure_auth.db")
    cursor = conn.cursor()

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE NOT NULL,
        email TEXT UNIQUE NOT NULL,
        password_hash TEXT NOT NULL,
        totp_secret TEXT,
        is_2fa_enabled BOOLEAN DEFAULT 0,
        backup_codes TEXT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    """)

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS sessions (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        session_token TEXT UNIQUE NOT NULL,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        expires_at TIMESTAMP NOT NULL
    )
    """)

    conn.commit()
    conn.close()

# =================
# PASSWORD MANAGER
# =================

def hash_password(password):
    salt = bcrypt.gensalt()
    return bcrypt.hashpw(password.encode('utf-8'), salt).decode('utf-8')

def verify_password(password, hashed):
    return bcrypt.checkpw(password.encode('utf-8'), hashed.encode('utf-8'))

def validate_password(password):
    """Validate password strength"""
    errors = []

    if len(password) < 8:
        errors.append("Password must be at least 8 characters")
    if not any(c.isupper() for c in password):
        errors.append("Password must contain uppercase letters")
    if not any(c.islower() for c in password):
        errors.append("Password must contain lowercase letters")
    if not any(c.isdigit() for c in password):
        errors.append("Password must contain digits")
    if not any(c in "!@#$%^&*()_+-=[]{}|;:,.<>?" for c in password):
        errors.append("Password must contain special characters")

    return len(errors) == 0, errors

# =============
# TOTP MANAGER
# =============

def generate_totp_secret():
    """Generate secure random TOTP secret"""
    return pyotp.random_base32()

def generate_backup_codes(count=10):
    """Generate backup recovery codes"""
    return [secrets.token_hex(4).upper() for _ in range(count)]

def display_qr_code(secret, username):
    """Generate and display QR code for authenticator app"""
    totp = pyotp.TOTP(secret)
    uri = totp.provisioning_uri(name=username, issuer_name="SecureApp")

    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4
    )
    qr.add_data(uri)
    qr.make(fit=True)

    qr_image = qr.make_image(fill_color="black", back_color="white")
    display(qr_image)
    time.sleep(0.5)

def verify_totp_code(secret, code):
    """Verify TOTP code with time window tolerance"""
    if not code or not code.isdigit() or len(code) != 6:
        return False

    totp = pyotp.TOTP(secret)
    return totp.verify(code, valid_window=1)

# ==================
# USER REGISTRATION
# ==================

def register_user():
    """Register new user with 2FA setup"""
    print("\n" + "="*60)
    print("USER REGISTRATION")
    print("="*60)

    # Get username
    username = input("Enter username: ").strip()

    conn = sqlite3.connect("secure_auth.db")
    cursor = conn.cursor()
    cursor.execute("SELECT id FROM users WHERE username = ?", (username,))

    if cursor.fetchone():
        print("ERROR: Username already exists")
        conn.close()
        return False

    conn.close()

    # Get email
    email = input("Enter email: ").strip()

    conn = sqlite3.connect("secure_auth.db")
    cursor = conn.cursor()
    cursor.execute("SELECT id FROM users WHERE email = ?", (email,))

    if cursor.fetchone():
        print("ERROR: Email already registered")
        conn.close()
        return False

    conn.close()

    # Get password
    password = input("Enter password (min 8 chars, uppercase, lowercase, digit, special): ").strip()

    # Validate password
    is_valid, errors = validate_password(password)
    if not is_valid:
        print("ERROR: Password does not meet requirements:")
        for error in errors:
            print("  - " + error)
        return False

    confirm = input("Confirm password: ").strip()

    if password != confirm:
        print("ERROR: Passwords do not match")
        return False

    # Hash password and generate TOTP secret
    password_hash = hash_password(password)
    totp_secret = generate_totp_secret()
    backup_codes = generate_backup_codes()

    # Save user to database
    conn = sqlite3.connect("secure_auth.db")
    cursor = conn.cursor()

    try:
        cursor.execute("""
        INSERT INTO users (username, email, password_hash, totp_secret, backup_codes)
        VALUES (?, ?, ?, ?, ?)
        """, (username, email, password_hash, totp_secret, json.dumps(backup_codes)))
        conn.commit()
        print("\nSUCCESS: User account created")

    except sqlite3.IntegrityError:
        print("ERROR: Username or email already exists")
        conn.close()
        return False

    conn.close()

    # Setup 2FA
    print("\n" + "="*60)
    print("SETUP TWO-FACTOR AUTHENTICATION")
    print("="*60)

    print("\nSecret Key: " + totp_secret)
    print("\nInstructions:")
    print("1. Open Google Authenticator app on your phone")
    print("2. Tap the plus (+) button")
    print("3. Select 'Scan QR code'")
    print("4. Point your camera at the QR code below")
    print("5. Wait for the account to be added")
    print("6. You will see a 6-digit code")

    print("\nQR CODE:")
    print("="*60)

    display_qr_code(totp_secret, username)

    print("="*60)
    print("\nIMPORTANT: After scanning, wait for the code to appear")
    print("in your Google Authenticator app (takes a few seconds)")
    print("\n" + "-"*60)
    print("Ready to verify? Enter the 6-digit code from your app:")
    print("-"*60 + "\n")

    # Verify TOTP code
    verification_code = input("Enter 6-digit code: ").strip()

    print()

    if verify_totp_code(totp_secret, verification_code):
        # Enable 2FA
        conn = sqlite3.connect("secure_auth.db")
        cursor = conn.cursor()
        cursor.execute("UPDATE users SET is_2fa_enabled = 1 WHERE username = ?", (username,))
        conn.commit()
        conn.close()

        print("SUCCESS: Code verified!")
        print("\n" + "="*60)
        print("TWO-FACTOR AUTHENTICATION ENABLED")
        print("="*60)

        print("\nBACKUP CODES (Save these securely!):")
        print("-"*60)
        for i, code in enumerate(backup_codes, 1):
            print("  {0:2d}. {1}".format(i, code))

        print("\n" + "-"*60)
        print("IMPORTANT:")
        print("  - Screenshot or copy these backup codes")
        print("  - Save them in a secure location")
        print("  - Each code can only be used ONCE")
        print("  - Use them if you lose access to your authenticator app")
        print("-"*60)

        print("\nRegistration complete!")
        print("You can now login with username, password, and 2FA code")

        return True
    else:
        print("ERROR: Invalid verification code")
        print("The code may have expired (codes change every 30 seconds)")
        return False

# ===========
# USER LOGIN
# ===========

def login_user():
    """Login user with 2FA verification"""
    print("\n" + "="*60)
    print("USER LOGIN")
    print("="*60)

    username = input("Username: ").strip()
    password = input("Password: ").strip()

    conn = sqlite3.connect("secure_auth.db")
    cursor = conn.cursor()

    cursor.execute("""
    SELECT id, password_hash, totp_secret, is_2fa_enabled, backup_codes
    FROM users WHERE username = ?
    """, (username,))

    user = cursor.fetchone()

    if not user:
        print("ERROR: Invalid username or password")
        conn.close()
        return None

    user_id, password_hash, totp_secret, is_2fa_enabled, backup_codes = user

    # Verify password
    if not verify_password(password, password_hash):
        print("ERROR: Invalid username or password")
        conn.close()
        return None

    print("SUCCESS: Password verified")

    # 2FA verification
    if is_2fa_enabled:
        print("\nTwo-Factor Authentication Required")
        print("-"*40)
        print("Check your Google Authenticator app")
        print("Enter the 6-digit code (changes every 30 seconds)\n")

        otp_code = input("Enter 6-digit code: ").strip()

        # Verify TOTP code
        if verify_totp_code(totp_secret, otp_code):
            print("SUCCESS: 2FA verified")
        else:
            # Check backup codes
            codes_list = json.loads(backup_codes) if backup_codes else []
            if otp_code.upper() in codes_list:
                codes_list.remove(otp_code.upper())
                cursor.execute("UPDATE users SET backup_codes = ? WHERE id = ?",
                             (json.dumps(codes_list), user_id))
                print("SUCCESS: Backup code accepted ({0} remaining)".format(len(codes_list)))
            else:
                print("ERROR: Invalid 2FA code")
                conn.close()
                return None

    # Create session
    session_token = secrets.token_urlsafe(32)
    expires_at = datetime.now() + timedelta(hours=1)

    cursor.execute("""
    INSERT INTO sessions (user_id, session_token, expires_at)
    VALUES (?, ?, ?)
    """, (user_id, session_token, expires_at))

    conn.commit()
    conn.close()

    print("\n" + "="*60)
    print("LOGIN SUCCESSFUL")
    print("="*60)
    print("Welcome, {0}!".format(username))
    print("\nSession Token: {0}...".format(session_token[:30]))
    print("Session expires in 1 hour")

    return session_token

# ==============
# CHECK SESSION
# ==============

def check_session(session_token):
    """Validate user session"""
    if not session_token:
        print("ERROR: No active session. Please login first.")
        return False

    conn = sqlite3.connect("secure_auth.db")
    cursor = conn.cursor()

    cursor.execute("""
    SELECT s.expires_at, u.username, u.email
    FROM sessions s
    JOIN users u ON s.user_id = u.id
    WHERE s.session_token = ?
    """, (session_token,))

    result = cursor.fetchone()
    conn.close()

    if not result:
        print("ERROR: Invalid session")
        return False

    expires_at_str, username, email = result
    expires_dt = datetime.fromisoformat(expires_at_str)

    if datetime.now() > expires_dt:
        print("ERROR: Session expired")
        return False

    print("\n" + "="*60)
    print("SESSION VALID")
    print("="*60)
    print("User: {0}".format(username))
    print("Email: {0}".format(email))
    print("Expires: {0}".format(expires_dt))

    return True

# ==========
# MAIN MENU
# ==========

def main():
    """Main application loop"""
    print("\nStarting 2FA Authentication System...\n")

    init_database()
    print("System initialized\n")

    current_session = None

    while True:
        print("\n" + "="*60)
        print("AUTHENTICATION SYSTEM")
        print("="*60)
        print("1. Register New User")
        print("2. Login")
        print("3. Check Session")
        print("4. Logout")
        print("5. Exit")
        print("-"*60)

        choice = input("Select option (1-5): ").strip()

        if choice == "1":
            register_user()

        elif choice == "2":
            session = login_user()
            if session:
                current_session = session

        elif choice == "3":
            check_session(current_session)

        elif choice == "4":
            if current_session:
                print("SUCCESS: Logged out successfully")
                current_session = None
            else:
                print("ERROR: Not logged in")

        elif choice == "5":
            print("\nGoodbye! Stay secure!\n")
            break

        else:
            print("ERROR: Invalid option. Please select 1-5.")

# ============
# RUN PROGRAM
# ============

if __name__ == "__main__":
    main()
